###### Trabajo relaizado en google colab , aprovechando su rapidez a la hora de ejecutar los codigos  de el webscrapeo

In [ ]:
# !pip install google.colab 

In [ ]:
from google.colab import drive  #importamos esta libreria que conecta el drive con el colab

El siguiente codigo es una especie  de permiso de acceder a nuestro drive para que haga uso de nuestros archivos
pasos para acceder al drive  
1-AL correr el codigo aparecera una ventana donde espera que coloquemos un permiso para continuar 
2 darle click al enlace que aparece 
3-selecionar la direcion de correo electronico que se usara para acceder al drive 
4-automaticamente nos saldra un enlace copiar y pegar en la ventana que aparece cuando corrimos el codigo


In [ ]:
drive.mount('/content/drive')  

In [ ]:
from bs4 import BeautifulSoup #importamos la libreria que nos ayudara con la extraccion de datos de la pagina unitprotein
import requests 
import pandas as pd #para el manejo de data frame
import numpy as np 
from tqdm.notebook import tqdm_notebook as tqdm #importamos esta libreria nueva para medir,validar el rrecorrido de la extraccion 

In [ ]:
ruta='/content/drive/MyDrive/webs/Genes identificados con vecinos.xlsx' #leo el archivo donde estan todos los genees